In [1]:
# testing out issues with mps
import torch
import platform
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")


print("Python:", platform.python_version())
print("macOS:", platform.mac_ver())
print("Machine:", platform.machine())
print("MPS Built:", torch.backends.mps.is_built())
print("MPS Available:", torch.backends.mps.is_available())
print("Torch Version:", torch.__version__)


Python: 3.10.13
macOS: ('13.6', ('', '', ''), 'arm64')
Machine: arm64
MPS Built: True
MPS Available: True
Torch Version: 2.2.2


In [ ]:
#loading packages and our data
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
       

# Load the data
# bring in lastfm data
import json

parada= pd.json_normalize([json.loads(line) for line in open('data/parada_full_dataset.json')])
parada.groupby("genre").size().reset_index(name='counts').sort_values(by='counts', ascending=False)
parada = parada.drop(parada[parada['genre'] != 'pop'].index)

Skipping invalid JSON line.
Skipping invalid JSON line.
Skipping invalid JSON line.


In [3]:
# now LIWC, NRC, distilbert all together : time for Factor Analysis
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

parada_num = parada.select_dtypes(include='number')

# NEED TO LOOK AT THIS RE  NA VALUES 
parada_num_clean = parada_num.dropna() 
print(f"Rows before: {parada_num.shape[0]}, after dropna: {parada_num_clean.shape[0]}")
parada_num_clean = parada_num_clean.loc[:, parada_num_clean.std() != 0]
print((parada_num_clean.std() == 0).sum()) 


chi_square_value,p_value=calculate_bartlett_sphericity(parada_num_clean)
chi_square_value, p_value

# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FactorAnalysis.html
# https://www.datacamp.com/tutorial/introduction-factor-analysis

from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(parada_num_clean)
print("kmo: ", kmo_model) # nan--issues



NameError: name 'parada' is not defined

In [ ]:
# too many columns and not ready for factor analysis
from sklearn.feature_selection import VarianceThreshold

# Remove columns with zero or near-zero variance
selector = VarianceThreshold(threshold=1e-5)
filtered_data = selector.fit_transform(parada_num_clean)

# Optionally get remaining column names
remaining_cols = parada_num_clean.columns[selector.get_support()]
parada_num_filtered = pd.DataFrame(filtered_data, columns=remaining_cols)
parada_num_filtered.drop("Segment", inplace=True, axis=1, errors='ignore')

#note had to edit error = sp. in factor analyzer.py to error = np. 


In [ ]:
# identify low-loading factors to remove
from factor_analyzer import FactorAnalyzer

# Perform Factor Analysis
fa = FactorAnalyzer(n_factors=2, rotation='varimax')
fa.fit(parada_num_filtered)

# Get factor loadings
loadings = fa.loadings_

# Set a threshold for removing low loadings
threshold = 0.5

# Identify columns (items) with low factor loadings
low_loading_items = [parada_num_clean.columns[i] for i in range(loadings.shape[0]) if all(abs(loadings[i, :]) < threshold)]

print(f"Items with low factor loadings (below {threshold}):")
print(low_loading_items)

# Remove those items from the original DataFrame
parada_num_concise = parada_num_clean.drop(columns=low_loading_items)

# Check the cleaned dataset
print(f"DataFrame after removing low loading items: {parada_num_concise.shape}")

print(f"Remaining columns after removing low loading items: {parada_num_concise.columns.tolist()}", f"Num of remaining columns: {parada_num_concise.shape[1]}")


In [ ]:
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Set threshold and number of factors
threshold = 0.25
n_factors = 2

# Standardize the data
scaler = StandardScaler()
parada_scaled = scaler.fit_transform(parada_num_filtered)

# Convert back to DataFrame to keep column names
parada_scaled_df = pd.DataFrame(parada_scaled, columns=parada_num_filtered.columns)

# Work on a copy for iterative elimination
parada_num_concise = parada_scaled_df.copy()

for round_num in range(1, 8):  # Run 7 rounds
    print(f"\n--- Round {round_num} ---")

    # Fit Factor Analysis
    fa = FactorAnalyzer(n_factors=n_factors, rotation='varimax')
    fa.fit(parada_num_concise)

    # Get loadings
    loadings = fa.loadings_

    # Identify low-loading items
    low_loading_items = [
        parada_num_concise.columns[i]
        for i in range(loadings.shape[0])
        if all(abs(loadings[i, :]) < threshold)
    ]

    print(f"Low-loading items (below {threshold}): {low_loading_items}")

    # Drop low-loading items
    parada_num_concise.drop(columns=low_loading_items, inplace=True)

    print( f"Num of remaining columns: {parada_num_concise.shape[1]}", f"Remaining columns after removing low loading items: {parada_num_concise.columns.tolist()}")





In [ ]:
# determine the number of factors
# 
import matplotlib.pyplot as plt

# Fit FactorAnalyzer with max possible factors
fa = FactorAnalyzer(n_factors=parada_num_concise.shape[1], rotation='varimax')
fa.fit(parada_num_concise)

# Get eigenvalues and plot them
ev, _ = fa.get_eigenvalues()

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(ev)+1), ev, marker='o', linestyle='-')
plt.axhline(1, color='red', linestyle='--', label='Eigenvalue = 1')
plt.title('Scree Plot for Factor Analysis')
plt.xlabel('Number of Factors')
plt.ylabel('Eigenvalue')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Prepare for Factor Analysis on the smaller set of columns
# redo our calculations:#
parada_num_concise = parada_num_clean[['features.entropy', 'affective.NRCExpandedEmotion.Anger', 'features.readability.flesch_reading_ease', 
                                       'affective.LIWC.total_word_count', 'affective.LIWC.ppron']].dropna()
#print(parada_num_concise.corr())
parada_num_concise = parada_num_concise.dropna()
#parada_num_concise.drop("Segment", axis=1, inplace=True)

#does it do better than identity matrix?
chi_square_value, p_value = calculate_bartlett_sphericity(parada_num_concise)
print("Bartlett’s Test: p =", p_value, "Chi-square =", chi_square_value)

# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FactorAnalysis.html
# https://www.datacamp.com/tutorial/introduction-factor-analysis

#KMO calculations
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(parada_num_concise)
kmo_model # 0.6 is not very good, 0.7 is acceptable, 0.8 is good, 0.9 is great
# https://bookdown.org/luguben/EFA_in_R/kaiser-meyer-olkin-kmo.html

In [ ]:
# # determine the number of factors
# # 
# import matplotlib.pyplot as plt

# # Fit FactorAnalyzer with max possible factors
# fa = FactorAnalyzer(n_factors=parada_num_concise.shape[1], rotation='varimax')
# fa.fit(parada_num_concise)

# # Get eigenvalues and plot them
# ev, _ = fa.get_eigenvalues()

# plt.figure(figsize=(8, 5))
# plt.plot(range(1, len(ev)+1), ev, marker='o', linestyle='-')
# plt.axhline(1, color='red', linestyle='--', label='Eigenvalue = 1')
# plt.title('Scree Plot for Factor Analysis')
# plt.xlabel('Number of Factors')
# plt.ylabel('Eigenvalue')
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [ ]:
# ready for factor analysis:
# redo but limit to 2 factors
fa = FactorAnalyzer(n_factors=2, rotation='varimax') 
fa.fit(parada_num_concise)

# Get the variance explained
variance, proportion_variance, cumulative_variance = fa.get_factor_variance()

# Print variance results
print("Variance per factor:", variance)
print("Proportion of variance per factor:", proportion_variance)
print("Cumulative variance (proportion):", cumulative_variance)


In [ ]:
# look at loadings
import seaborn as sns

loadings = fa.loadings_

# Create a DataFrame for factor loadings for easier visualization
loadings_df = pd.DataFrame(loadings, columns=[f'Factor {i+1}' for i in range(loadings.shape[1])], index=parada_num_concise.columns)
sorted_loadings_df = loadings_df.sort_values(by='Factor 2', ascending=False)


# Print factor loadings for each variable
print([sorted_loadings_df.head(10)])

# Plot bar chart of loadings for each variable in each factor
plt.figure(figsize=(10, 6))
# Set the axis limits for zooming in
sorted_loadings_df.plot(kind='bar', figsize=(12, 8))
plt.title("Factor Loadings for Each Variable")
plt.xlabel("Variables")
plt.ylabel("Factor Loadings")
plt.xticks(rotation=90)  # Rotate variable names for readability
plt.tight_layout()
plt.show()




In [ ]:
# look more at loadings and results: biplot
# biplot
import matplotlib.pyplot as plt
factor_scores = fa.transform(parada_num_concise)

# Create a biplot: Factor scores (observations) and factor loadings (variables)
plt.figure(figsize=(10, 8))

# Plot factor scores
sns.scatterplot(x=factor_scores[:, 0], y=factor_scores[:, 1], alpha=0.6, label='Observations')

# Plot factor loadings (arrows for variables)
for i, var in enumerate(loadings_df.index):
    plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], color='red', alpha=0.7, head_width=0.05)
    plt.text(loadings[i, 0] * 1.1, loadings[i, 1] * 1.1, var, color='black', ha='center', va='center', fontweight='bold')

# Labels and title
plt.xlabel('Factor 1')
plt.ylabel('Factor 2')
# Set the axis limits for zooming in
plt.xlim(-1, 1)
plt.ylim(-2, 2)
plt.title('Biplot of Factor Scores and Factor Loadings')
plt.axhline(0, color='gray',linewidth=0.5)
plt.axvline(0, color='gray',linewidth=0.5)
plt.grid(True)
plt.tight_layout()
plt.show()


## K-Means: clustering of factor results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Range of cluster sizes to evaluate
k_range = range(2, 18)

# Store inertia and silhouette scores
inertias = []
silhouette_scores = []

# Fit KMeans for each k and record metrics
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=13)
    kmeans.fit(factor_scores)
    inertias.append(kmeans.inertia_)
    labels = kmeans.labels_
    sil_score = silhouette_score(factor_scores, labels)
    silhouette_scores.append(sil_score)

# Plot both inertia and silhouette score
fig, ax1 = plt.subplots(figsize=(10, 6))

# Inertia plot
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia', color='tab:blue')
ax1.plot(k_range, inertias, marker='o', label='Inertia', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Silhouette score plot (secondary axis)
ax2 = ax1.twinx()
ax2.set_ylabel('Silhouette Score', color='tab:green')
ax2.plot(k_range, silhouette_scores, marker='s', linestyle='--', label='Silhouette Score', color='tab:green')
ax2.tick_params(axis='y', labelcolor='tab:green')

plt.title('KMeans: Inertia and Silhouette Score by Number of Clusters')
plt.grid(True)
fig.tight_layout()
plt.show()

# Optional: Print both metrics in tabular form
print("k\tInertia\t\tSilhouette Score")
for i, k in enumerate(k_range):
    print(f"{k}\t{inertias[i]:.2f}\t\t{silhouette_scores[i]:.3f}")


In [ ]:
# commit to clusters
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

kmeans_4 = KMeans(n_clusters=13, random_state=13)
clusters4 = kmeans_4.fit_predict(factor_scores)



In [ ]:
# visualize clusters

# Convert factor scores to a DataFrame if it's not already
factor_df = pd.DataFrame(factor_scores, columns=[f'Factor {i+1}' for i in range(factor_scores.shape[1])])
factor_df['Cluster'] = clusters4  # Add KMeans cluster labels

# Plot first two factors
plt.figure(figsize=(10, 7))
sns.scatterplot(
    x='Factor 1', y='Factor 2',
    hue='Cluster',
    palette='tab10',
    data=factor_df,
    alpha=0.7,
    s=40
)
plt.title("KMeans Clusters on First Two Factor Scores")
plt.xlabel("Factor 1")
plt.ylabel("Factor 2")
plt.legend(title='Cluster')
plt.grid(True)
plt.tight_layout()
plt.show()


# # Visualize with colors
# sns.scatterplot(x=umap_embedding[:, 0], y=umap_embedding[:, 1], hue=clusters5, palette="viridis", s=30)
# plt.title("UMAP of Factor Scores with Clusters")
# plt.show()

In [ ]:
print(parada_num_filtered.columns)

In [ ]:
from bertopic import BERTopic

# Step 1: Attach cluster labels to parada dataframe, ensuring alignment
parada_t =  parada_num_concise.join(parada['lyrics'], how='inner')
#parada_t['lyrics'] = parada['lyrics'].dropna()
#parada_t = parada.reset_index(drop=True)  # Ensure matching index
parada_t['Cluster'] = clusters4        # Add clusters to main dataframe

# Step 2: Initialize dictionary to store topic models per cluster
cluster_topic_models = {}

# Step 3: Loop through clusters and apply BERTopic
for cluster_id in sorted(parada_t['Cluster'].unique()):
    cluster_lyrics = parada_t[parada_t['Cluster'] == cluster_id]['lyrics'].dropna().astype(str).tolist()

    if len(cluster_lyrics) > 10:  # Skip clusters with very few entries
        print(f"\nFitting topic model for Cluster {cluster_id} with {len(cluster_lyrics)} documents...")
        topic_model = BERTopic()
        topics, probs = topic_model.fit_transform(cluster_lyrics)
        cluster_topic_models[cluster_id] = topic_model

        # Optionally print top topic
        print(topic_model.get_topic_info().head(3))


In [ ]:
# from sklearn.neighbors import NearestNeighbors
# import matplotlib.pyplot as plt
# import numpy as np

# neigh = NearestNeighbors(n_neighbors=5)
# nbrs = neigh.fit(parada_num_concise.drop(columns=['dbscan_cluster'], errors='ignore'))
# distances, indices = nbrs.kneighbors(parada_num_concise.drop(columns=['dbscan_cluster'], errors='ignore'))

# distances = np.sort(distances[:, 4])  # 4th NN
# plt.plot(distances)
# plt.axhline(y=0.5, color='r', linestyle='--')  # current eps
# plt.title("K-distance Plot (4th NN)")
# plt.xlabel("Data Points sorted by distance")
# plt.ylabel("4th Nearest Neighbor Distance")
# plt.grid(True)
# plt.show()


In [ ]:
# # Export as df:

# # Step 1: Filter out rows with missing values in keeper columns
# parada_clusters = parada[parada[['Cognition', 'Linguistic', 'swear', 'emo_pos']].notna().all(axis=1)].copy()

# # Step 2: Add the cluster labels
# parada_clusters['cluster'] = clusters5

# # Step 3: Create the factor scores DataFrame (factor_scores should be aligned with the same filtered rows)
# factor_df = pd.DataFrame(
#     factor_scores,
#     columns=[f'Factor {i+1}' for i in range(factor_scores.shape[1])],
#     index=parada_clusters.index  # <- Use parada_clusters index here for perfect alignment
# )

# # Step 4: Concatenate factor scores into parada_clusters
# parada_clusters = pd.concat([parada_clusters, factor_df], axis=1)

# # Step 5: Check the columns
# print(parada_clusters.columns.tolist())

# # Step 6: Load the CSV file and keep only the first two columns
# songs_df = pd.read_csv('data/songs_expanded_25_04_01.csv', usecols=[0, 1])

# # Step 7: Ensure index alignment — check and align if needed
# # Option 1: If index matches parada_clusters
# # parada_clusters = parada_clusters.merge(songs_df, left_index=True, right_index=True)

# # Option 2: If there's a common key column (e.g., 'song_id' or 'title'), merge on that
# # Replace 'key_column' with the actual shared column name
# # parada_clusters = parada_clusters.merge(songs_df, on='key_column', how='left')

# # Step 7 (recommended): Add the two columns directly, assuming order matches
# parada_clusters = parada_clusters.reset_index(drop=True)  # Ensure matching index
# songs_df = songs_df.reset_index(drop=True)

# parada_clusters = pd.concat([songs_df, parada_clusters], axis=1)

# # Step 8: Check final DataFrame
# print(parada_clusters.head())


In [ ]:
# ## UMAP
# import umap
# import matplotlib.pyplot as plt

# # Initialize UMAP
# for n in [5, 15, 30, 50]:
#     embedding = umap.UMAP(n_neighbors=n, random_state=13).fit_transform(parada_num_concise)
#     plt.scatter(embedding[:, 0], embedding[:, 1], s=5)
#     plt.title(f'n_neighbors={n}')
#     plt.show()

# # Fit and transform the factor scores
# # umap_embedding = reducer.fit_transform(factor_scores)




In [ ]:
# import numpy as np
# import pandas as pd
# import umap
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score
# import matplotlib.pyplot as plt

# # Set up values to test
# neighbors_range = [5,  50, 75]
# silhouette_scores = []

# for n in neighbors_range:
#     # Step 1: UMAP projection
#     reducer = umap.UMAP(n_neighbors=n, n_components=2, random_state=13)
#     embedding = reducer.fit_transform(factor_scores)  # or your high-dim data

#     # Step 2: Clustering (adjust k as needed)
#     kmeans = KMeans(n_clusters=5, random_state=13)
#     labels = kmeans.fit_predict(embedding)

#     # Step 3: Silhouette score
#     score = silhouette_score(embedding, labels)
#     silhouette_scores.append(score)

# # Step 4: Plot the results
# plt.figure(figsize=(8, 5))
# plt.plot(neighbors_range, silhouette_scores, marker='o')
# plt.title("Silhouette Score vs n_neighbors")
# plt.xlabel("UMAP n_neighbors")
# plt.ylabel("Silhouette Score")
# plt.grid(True)
# plt.show()


In [ ]:
# from umap import UMAP
# factor_cols = [col for col in parada_clusters.columns if col.startswith('Factor')]

# # Step 2: Generate UMAP embeddings
# umap_model = UMAP(random_state=13)
# umap_embedding = umap_model.fit_transform(parada_clusters[factor_cols])

# # Step 3: Plot the scatterplot with hue by cluster
# plt.figure(figsize=(12, 8))
# sns.scatterplot(
#     x=umap_embedding[:, 0],
#     y=umap_embedding[:, 1],
#     hue=parada_clusters['cluster'],
#     palette='viridis',
#     s=30,
#     alpha=0.8,
#     edgecolor='none'
# )

# # Step 4: Add artist labels (optional: limit to avoid clutter)
# for i, artist in enumerate(parada_clusters.iloc[:10, 0]):  # 
#     plt.text(
#         umap_embedding[i, 0],
#         umap_embedding[i, 1],
#         str(artist),
#         fontsize=6,
#         alpha=0.6
#     )

# plt.title("UMAP of Factor Scores with Clusters and Artist Labels")
# plt.xlabel("UMAP Dimension 1")
# plt.ylabel("UMAP Dimension 2")
# plt.legend(title='Cluster')
# plt.tight_layout()
# plt.show()

In [ ]:
# print(parada_clusters.columns.tolist())

In [ ]:
# parada_clusters.to_csv('data/songs_expandedc_25_05_12.csv', index=False)



In [ ]:
# parada_clusters.describe(include='object')
# print(parada_clusters.columns.tolist())

In [ ]:
# parada_clusters = pd.read_csv('data/songs_expandedc_25_05_12.csv') 
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

# keepers = ["function", "Cognition", "cogproc", "Linguistic", "WC","Comma", "swear","Analytic",
# "Social", "socrefs", "pronoun", "ppron", "space", "Perception", "article"]
# parada_clusters = pd.get_dummies(parada_clusters, columns=['emotion'], prefix='emotion')
# parada_clusters = pd.get_dummies(parada_clusters, columns=['artist'], drop_first=False)
# parada_clusters= parada_clusters.dropna()
# drops = keepers + ['album','cluster', 'lyrics', 'song', 'Tone','lyrics-tokenized','emotion_scores_nrc', 'Factor 1', 'Factor 2']
# #X = parada_clusters.drop(columns=drops)       # Features (e.g. factor scores, PCA, etc.)
# X = parada_clusters[['surprise', 'joy', 'anger', 'neutral', 'sadness', 'fear', 'disgust', 'anger_nrc', 'fear_nrc', 'joy_nrc', 'sadness_nrc', 'surprise_nrc', 'disgust_nrc', 'trust_nrc', 'anticipation_nrc', 'negative_nrc', 'positive_nrc']]
# y = parada_clusters['cluster']    


# # Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

# # Train model
# clf = RandomForestClassifier(random_state=13)
# clf.fit(X_train, y_train)

# # Predict on test set
# y_pred = clf.predict(X_test)

# # Evaluate performance
# print(classification_report(y_test, y_pred))

In [ ]:
# importances = clf.feature_importances_
# features = X_train.columns

# # Create a Series for easy visualization
# feat_importance = pd.Series(importances, index=features).sort_values(ascending=False)

# # Top 10 important features
# feat_importance.head(10).plot(kind='barh')
# plt.title("Top 10 Feature Importances")
# plt.gca().invert_yaxis()
# plt.show()